In [1]:
import pandas as pd
import os

GPU               = '1'
os.environ['CUDA_VISIBLE_DEVICES'] = GPU

import numpy as np
from gensim.models import Word2Vec, FastText
import glove
from glove import Corpus
import collections
import gc 
import keras
from keras import backend as K
from keras import regularizers
from keras.models import Sequential, Model
from keras.layers import Flatten, Dense, Dropout, Input, concatenate, Concatenate, Activation, Concatenate, LSTM, GRU
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Conv1D, BatchNormalization, GRU, Convolution1D, LSTM
from keras.layers import UpSampling1D, MaxPooling1D, GlobalMaxPooling1D, GlobalAveragePooling1D,MaxPool1D, Concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint, History, ReduceLROnPlateau
from keras.utils import np_utils
# from keras.backend.tensorflow_backend import set_session, clear_session, get_session
from keras.backend import set_session, clear_session, get_session
import tensorflow as tf
from keras.optimizers import Adam
from sklearn.utils import class_weight
from sklearn.metrics import average_precision_score, roc_auc_score, accuracy_score, f1_score
import warnings
warnings.filterwarnings('ignore')
import time
def mean(a):
    return sum(a) / len(a)



2023-05-01 00:00:34.747858: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-01 00:00:34.798223: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-01 00:00:34.799199: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-01 00:00:35.495938: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Reset Keras Session
def reset_keras(model):
    sess = get_session()
    clear_session()
    sess.close()
    sess = get_session()

    try:
        del model # this is from global space - change this as you need
    except:
        pass

    gc.collect() # if it's done something you should see a number being outputted

def create_dataset(dict_of_ner):
    temp_data = []
    for k, v in sorted(dict_of_ner.items()):
        temp = []
        for embed in v:
            temp.append(embed)
        temp_data.append(np.mean(temp, axis = 0)) 
    return np.asarray(temp_data)

def make_prediction_multi_avg(model, test_data):
    probs = model.predict(test_data)
    y_pred = [1 if i>=0.5 else 0 for i in probs]
    return probs, y_pred

def save_scores_multi_avg(predictions, probs, ground_truth, 
                          
                          embed_name, problem_type, iteration, hidden_unit_size,
                          
                          sequence_name, type_of_ner):
    
    auc = roc_auc_score(ground_truth, probs)
    auprc = average_precision_score(ground_truth, probs)
    acc   = accuracy_score(ground_truth, predictions)
    F1    = f1_score(ground_truth, predictions)
    
    result_dict = {}    
    result_dict['auc'] = auc
    result_dict['auprc'] = auprc
    result_dict['acc'] = acc
    result_dict['F1'] = F1
    
    result_path = "results/multimodal-baseline"
    file_name = str(sequence_name)+"-"+str(hidden_unit_size)+"-"+embed_name
    file_name = file_name +"-"+problem_type+"-"+str(iteration)+"-"+type_of_ner+"-avg-.p"
    pd.to_pickle(result_dict, os.path.join(result_path, file_name))

    print(auc, auprc, acc, F1)
    
def avg_ner_model(layer_name, number_of_unit, embedding_name):

    if embedding_name == "concat":
        input_dimension = 200
    elif embedding_name == "biobert": 
        input_dimension = 768
    else:
        input_dimension = 100

    sequence_input = Input(shape=(24,104))

    input_avg = Input(shape=(input_dimension, ), name = "avg")        
#     x_1 = Dense(256, activation='relu')(input_avg)
#     x_1 = Dropout(0.3)(x_1)
    
    if layer_name == "GRU":
        x = GRU(number_of_unit)(sequence_input)
    elif layer_name == "LSTM":
        x = LSTM(number_of_unit)(sequence_input)

    x = keras.layers.Concatenate()([x, input_avg])

    x = Dense(256, activation='relu')(x)
    x = Dropout(0.2)(x)
    
    
#     logits_regularizer = tf.contrib.layers.l2_regularizer(scale=0.01)
    
#     preds = Dense(1, activation='sigmoid',use_bias=False,
#                          kernel_initializer=tf.contrib.layers.xavier_initializer(), 
#                   kernel_regularizer=logits_regularizer)(x)
    
    
# https://stackoverflow.com/questions/70447036/run-the-code-tensorflow-1-15-in-tensorflow-2-7
    logits_regularizer = tf.keras.regularizers.l2(l=0.5 * (0.01))
    preds = Dense(1, activation='sigmoid',use_bias=False,kernel_initializer=tf.compat.v1.keras.initializers.glorot_normal(), 
                  kernel_regularizer=logits_regularizer)(x)
    
    adam_opt = Adam(lr=0.0006, decay=0.01)
        
    model = Model(inputs=[sequence_input, input_avg], outputs=preds)
    model.compile(loss='binary_crossentropy',
                  optimizer=adam_opt,
                  metrics=['acc'])
    
    return model

In [3]:
type_of_ner = "new"

x_train_lstm = pd.read_pickle("data/"+type_of_ner+"_x_train.pkl")
x_dev_lstm = pd.read_pickle("data/"+type_of_ner+"_x_dev.pkl")
x_test_lstm = pd.read_pickle("data/"+type_of_ner+"_x_test.pkl")

y_train = pd.read_pickle("data/"+type_of_ner+"_y_train.pkl")
y_dev = pd.read_pickle("data/"+type_of_ner+"_y_dev.pkl")
y_test = pd.read_pickle("data/"+type_of_ner+"_y_test.pkl")

ner_word2vec = pd.read_pickle("data/"+type_of_ner+"_ner_word2vec_limited_dict.pkl")
ner_fasttext = pd.read_pickle("data/"+type_of_ner+"_ner_fasttext_limited_dict.pkl")
ner_concat = pd.read_pickle("data/"+type_of_ner+"_ner_combined_limited_dict.pkl")
# ner_biobert = pd.read_pickle("data/"+type_of_ner+"_ner_biobert_limited_dict.pkl")

ner_biobert = pd.read_pickle("data/biobert_test_data.pkl")

train_ids = pd.read_pickle("data/"+type_of_ner+"_train_ids.pkl")
dev_ids = pd.read_pickle("data/"+type_of_ner+"_dev_ids.pkl")
test_ids = pd.read_pickle("data/"+type_of_ner+"_test_ids.pkl")

In [4]:

monitor_criteria = 'val_loss'
model_patience = 5
batch_size = 64


# num_epoch = 100
target_problems = ['mort_hosp', 'mort_icu', 'los_3', 'los_7']
iter_num = 2
# layers = ["LSTM", "GRU"]
# unit_sizes = [128, 256]
embedding_types = ['word2vec', 'fasttext', 'concat','biobert']
embedding_dict = [ner_word2vec, ner_fasttext, ner_concat, ner_biobert]

#embedding_types = ['word2vec', 'fasttext', 'concat']
#embedding_dict = [ner_word2vec, ner_fasttext, ner_concat]


num_epoch = 70
layers = ["GRU"]
unit_sizes = [256]


total_model_time_initial = time.time()
embedding_timer = {}

for item in embedding_types:
    embedding_timer[item] = list()

index = 0

for each_layer in layers:
    print ("Layer: ", each_layer)
    for each_unit_size in unit_sizes:
        print ("Hidden unit: ", each_unit_size)

        for embed_dict, embed_name in zip(embedding_dict, embedding_types):
            print ("Embedding: ", embed_name)
            print("=============================")
            
            

            temp_train_ner = dict((k, embed_dict[k]) for k in train_ids)
            temp_dev_ner = dict((k, embed_dict[k]) for k in dev_ids)
            temp_test_ner = dict((k, embed_dict[k]) for k in test_ids)

            x_train_ner = create_dataset(temp_train_ner)
            x_dev_ner = create_dataset(temp_dev_ner)
            x_test_ner = create_dataset(temp_test_ner)


            for cycle in range(0, iter_num):
                iteration = cycle + 1
                print ("Iteration number: ", iteration)

                for each_problem in target_problems:
                    print ("Problem type: ", each_problem)
                    print ("__________________")
                    
                    initial_timer = time.time()

                    early_stopping_monitor = EarlyStopping(monitor=monitor_criteria, patience=model_patience)
                    best_model_name = "avg-"+str(embed_name)+"-"+str(each_problem)+"-"+"best_model.hdf5"
                    checkpoint = ModelCheckpoint(best_model_name, monitor='val_loss', verbose=1,
                        save_best_only=True, mode='min', save_freq='epoch')


                    callbacks = [early_stopping_monitor, checkpoint]

                    model = avg_ner_model(each_layer, each_unit_size, embed_name)

                    model.fit([x_train_lstm, x_train_ner], y_train[each_problem], epochs=num_epoch, verbose=1, 
                              validation_data=([x_dev_lstm, x_dev_ner], y_dev[each_problem]), callbacks=callbacks, 
                              batch_size=batch_size )
                
                    model.load_weights(best_model_name)

                    probs, predictions = make_prediction_multi_avg(model, [x_test_lstm, x_test_ner])
                    
                    save_scores_multi_avg(predictions, probs, y_test[each_problem], 
                               embed_name, each_problem, iteration, each_unit_size, 
                               each_layer, type_of_ner)
                    
                    reset_keras(model)
                    #del model
                    clear_session()
                    gc.collect()

                    embedding_timer[embed_name].append(time.time()-initial_timer)
                    with open(f"/home/ubuntu/times_{embed_name}_{each_problem}_iter_{iteration}.txt", "w") as file1:
                        file1.write(f"time to run {embed_name}-{each_problem}: {embedding_timer[embed_name][len(embedding_timer[embed_name]) -1]}")
print(f"final model training time: {time.time() - total_model_time_initial}")

Layer:  GRU
Hidden unit:  256
Embedding:  word2vec
Iteration number:  1
Problem type:  mort_hosp
__________________


2023-05-01 00:00:50.736522: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-01 00:00:50.738107: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-01 00:00:50.739203: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Epoch 1/70


2023-05-01 00:00:51.522392: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-01 00:00:51.524287: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-01 00:00:51.525578: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

241/242 [============================>.] - ETA: 0s - loss: 0.2774 - acc: 0.9057

...

241/242 [============================>.] - ETA: 0s - loss: 0.2682 - acc: 0.9210

2023-05-01 01:13:04.409151: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-01 01:13:04.410828: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-01 01:13:04.411991: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus


Epoch 1: val_loss improved from inf to 0.25806, saving model to avg-biobert-los_7-best_model.hdf5
242/242 [==============================] - 9s 32ms/step - loss: 0.2681 - acc: 0.9210 - val_loss: 0.2581 - val_acc: 0.9238
Epoch 2/70
241/242 [============================>.] - ETA: 0s - loss: 0.2501 - acc: 0.9225
Epoch 2: val_loss did not improve from 0.25806
242/242 [==============================] - 7s 30ms/step - loss: 0.2499 - acc: 0.9225 - val_loss: 0.2601 - val_acc: 0.9238
Epoch 3/70
241/242 [============================>.] - ETA: 0s - loss: 0.2468 - acc: 0.9223
Epoch 3: val_loss improved from 0.25806 to 0.25413, saving model to avg-biobert-los_7-best_model.hdf5
242/242 [==============================] - 7s 31ms/step - loss: 0.2468 - acc: 0.9223 - val_loss: 0.2541 - val_acc: 0.9238
Epoch 4/70
241/242 [============================>.] - ETA: 0s - loss: 0.2429 - acc: 0.9226
Epoch 4: val_loss did not improve from 0.25413
242/242 [==============================] - 7s 30ms/step - loss: 0.

2023-05-01 01:15:11.048975: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-01 01:15:11.050601: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-01 01:15:11.051781: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

138/138 [==============================] - 1s 9ms/step
0.744309367844309 0.24116557201659883 0.919872524470749 0.016759776536312852
final model training time: 4463.935860157013


In [5]:
# a

In [6]:
embedding_types = ['word2vec', 'fasttext', 'concat','biobert']

for item in embedding_types:
    print(f"{embedding_timer[item]}")
    
print(f"final model training time: {time.time() - total_model_time_initial}")

[154.21198153495789, 108.14102745056152, 131.40392804145813, 199.23999428749084, 153.30717134475708, 121.70263600349426, 159.22947239875793, 194.98438954353333]
[106.10375833511353, 77.84544157981873, 112.21864557266235, 159.15052151679993, 78.17005109786987, 136.94606518745422, 121.70871949195862, 114.63378572463989]
[100.16451692581177, 150.883043050766, 201.45986557006836, 158.65079617500305, 146.9994649887085, 110.6108648777008, 125.10123825073242, 197.18419122695923]
[86.60719466209412, 102.06179594993591, 181.22611117362976, 152.58204054832458, 152.55921387672424, 145.80921125411987, 181.53472995758057, 138.50401306152344]
final model training time: 4463.946449756622


In [7]:
# a